In [27]:
import pandas as pd

analysis = 'FIPO'
df = pd.read_csv(f'/Users/lucamouchel/epfl/logical-fallacies-repo/Logical-Fallacies/src/EVAL/AMT/annotations/SFT_vs_{analysis}.csv')
df = df [['Input.topic', 'Input.stance', 'Input.argument1', 'Input.argument2', 'Input.sft_index', 'Answer.q1_valid']]
gpt_4_fallacies = pd.read_json(f'/Users/lucamouchel/epfl/logical-fallacies-repo/Logical-Fallacies/results/llama_bis/cpo_results/f-rate.json')
sft_gpt_4 = pd.read_json(f'/Users/lucamouchel/epfl/logical-fallacies-repo/Logical-Fallacies/results/llama/arguments/sft/f-rate.json')
set_topic = (df['Input.topic'].unique())
seen_topics = {topic: [] for topic in set_topic}
for i, entry in df.iterrows():
    seen_topics[entry['Input.topic']].append(entry['Answer.q1_valid'])

In [30]:
from collections import Counter
is_fallacy = []
preds = []


results = {'SFT': 0, analysis: 0, 'tie_good': 0, 'tie_bad': 0}
l, l2=0, 0
K=0
good, bad = 0,0
A,B=0,0
L = 0
comparisons = []

T , X = 0,0
total = 0
for k,v in seen_topics.items():
    
    sft_index = df[df['Input.topic'] == k].iloc[0]['Input.sft_index']
    counts = Counter(v)
    gpt_4 = gpt_4_fallacies[gpt_4_fallacies.topic==k].iloc[0]

    sft = sft_gpt_4[sft_gpt_4.topic==k].iloc[0]
    sft_f_type = sft.fallacy_type

    argument = gpt_4.argument
    f_type = gpt_4.fallacy_type

   
    if not all([count == 1 for count in counts.values()]):
        maximum_frequency = max(counts, key=counts.get)
        if maximum_frequency == -10:
            results['tie_bad'] += 1
            winner = 'tie'
        elif maximum_frequency == 10:
            results['tie_good'] += 1
            winner = 'tie'
        elif maximum_frequency in [1,2]:
            maximum_frequency -= 1
            if maximum_frequency != sft_index:
                results[analysis] += 1
                winner = analysis
            else:
                results['SFT'] += 1
                winner = 'SFT'

        if winner =='tie' :
            if f_type == 'None':
                T += 1
            else :
                X += 1
        if (winner == analysis):
            if f_type == 'None':
                l += 1
            else: 
                l2 +=1

        if (winner == 'SFT'):
            if sft_f_type == 'None':
                A += 1
            else :
                B += 1
        if winner==analysis and f_type == 'None':
            L += 1
        if winner == 'SFT' and f_type == 'None' and sft_f_type != 'None':
            K += 1

        if winner == 'SFT' and f_type == 'None':
            comparisons.append((argument, f_type, sft_f_type, winner))

print(f"% of times humans selected {analysis} and it was NOT considered a fallacy: ", round(l/(results[analysis]), 3))
print(f"% of times humans selected TIE and {analysis} was NOT considered a fallacy: ", round(T/(results['tie_good']+results['tie_bad']), 3))
print(f"% of times humans selected SFT and it was NOT considered a fallacy: ", round( A/results['SFT'], 3))
print(f"% of times human selected {analysis} when {analysis} was a fallacy ", round(L/len(seen_topics), 3))
print(f"% of times human selected SFT when {analysis} was not a fallacy and SFT was a fallacy: ", round(K/len(seen_topics),3))

% of times humans selected FIPO and it was NOT considered a fallacy:  0.821
% of times humans selected TIE and FIPO was NOT considered a fallacy:  0.844
% of times humans selected SFT and it was NOT considered a fallacy:  0.765
% of times human selected FIPO when FIPO was a fallacy  0.294
% of times human selected SFT when FIPO was not a fallacy and SFT was a fallacy:  0.037


In [29]:
results

{'SFT': 34, 'FIPO': 67, 'tie_good': 36, 'tie_bad': 9}

In [61]:
from collections import Counter
is_fallacy = []
preds = []


results = {'SFT': 0, analysis: 0, 'tie_good': 0, 'tie_bad': 0}

winners = []
miss =0 
import json
with open('/root/Logical-Fallacies/src/EVAL/AMT/winners.json', 'r') as f:
    gpt_4 = json.load(f)


i = 0
exact = 0
new_gpt_4 = []
sft_stuff = []
for  GPT_4, (k,v) in (zip(gpt_4, seen_topics.items())):

    sft_index = df[df['Input.topic'] == k].iloc[0]['Input.sft_index']
    counts = Counter(v)

    if not all([count == 1 for count in counts.values()]):
        new_gpt_4.append(GPT_4)
        
        maximum_frequency = max(counts, key=counts.get)
        if maximum_frequency == -10:
            results['tie_bad'] += 1
            winner = 'tie'
        elif maximum_frequency == 10:
            results['tie_good'] += 1
            winner = 'tie'
        elif maximum_frequency in [1,2]:
            maximum_frequency -= 1
            if maximum_frequency != sft_index:
                results[analysis] += 1
                winner = 'custom'
            else:
                results['SFT'] += 1
                winner = 'sft'
        sft_stuff.append(winner)
        if GPT_4 == 'custom' and winner == 'sft':
            miss += 1
        if winner == GPT_4:
            exact += 1
    i+=1
    

        

In [63]:
Counter(new_gpt_4)

Counter({'sft': 53, 'custom': 88, 'tie': 5})

In [64]:
num_agree = 0
for gpt, sft in zip(new_gpt_4, sft_stuff):
    num_agree += gpt == sft

In [65]:
num_agree

63

In [66]:
{'sft': 0, 'custom': 1, 'tie': 2}

new_gpt_4 = [1 if x == 'sft' else 2 if x == 'custom' else 3 for x in new_gpt_4]
sft_stuff = [1 if x == 'sft' else 2 if x == 'custom' else 3 for x in sft_stuff]

In [44]:
import numpy as np 

In [67]:
correlation = np.corrcoef(new_gpt_4, sft_stuff)[0, 1]
correlation

0.11517173326591945

In [39]:
num_agree

91

In [18]:
exact

91

In [26]:
results

{'SFT': 34, 'CPO_CUSTOM': 67, 'tie_good': 36, 'tie_bad': 9}

In [27]:
results

{'SFT': 34, 'CPO_CUSTOM': 67, 'tie_good': 36, 'tie_bad': 9}

In [303]:
l/results[analysis], A/results['SFT'], L/len(seen_topics), K/len(seen_topics)

(0.6666666666666666,
 0.7166666666666667,
 0.0855614973262032,
 0.058823529411764705)

For DPO. 79.5% of the time they select DPO as a winner, it is not a fallacy. So only 20% of the time they select DPO as a fallacy

Basically, we should detect when the humans have chosen a sample, if they select a fallacy statement. So basically, when they pick DPO for example, does gpt4 classify it as a fallacy? if yes, then it's bad, otherwise it's good and DPO truly is better. 
When they select SFT, do they select a fallacy statement?

In [304]:
print(l/len(seen_topics))
print(A/len(seen_topics))


0.26737967914438504
0.22994652406417113


Basically, we should penalize if it chose a fallacy statement.

In [305]:
results

{'SFT': 60, 'CPO': 75, 'tie_good': 7, 'tie_bad': 7}

In [306]:
d = {'SFT': results['SFT'], analysis: results[analysis], 'tie': results['tie_good'] + results['tie_bad']}

In [307]:
total

0

In [308]:
total = sum(d.values())
for k,v in d.items():
    print(f'{k}: {v} ({v/total*100:.2f}%)')

SFT: 60 (40.27%)
CPO: 75 (50.34%)
tie: 14 (9.40%)


In [3]:
import pandas as pd
pd.read_csv('/root/Logical-Fallacies/src/EVAL/AMT/rewardsaccuracies.csv')['misunderstood-firebrand-238 - train/rewards/accuracies']


0      0.03125
1      0.04375
2      0.06250
3      0.14375
4      0.16250
        ...   
138    1.00000
139    1.00000
140    1.00000
141    1.00000
142    1.00000
Name: misunderstood-firebrand-238 - train/rewards/accuracies, Length: 143, dtype: float64